In [1]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np 
import math
from tensorflow.keras.models import load_model 
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense,Dropout

In [3]:
dataset = mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #translate the data to image 

x_train = x_train.reshape(x_train.shape[0], -1).astype("float64") 
x_test = x_test.reshape(x_test.shape[0], -1).astype("float64")

#normalization 
x_train /= 255 
x_test /= 255

x_train = x_train.reshape(-1, 28, 28, 1) 
x_test = x_test.reshape(-1, 28, 28, 1)

        #translate y_train and y_test to “one hot” form
#y_train = utils.to_categorical(y_train) 
y_test = utils.to_categorical(y_test)
#print(y_test.shape)

lenet5 = load_model('Model/lenet5_mnist.h5')#load lenet5 model trained with mnist dataset 
lenet5.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [8]:
beta = 0.11299
neighbourlist,neighbourLabel = neighbour_image(x_train,y_train,beta)
#print(len(neighbourlist))

new_train = np.append(x_train,neighbourlist)
new_train = new_train.reshape(-1,28,28,1)
new_label = np.append(y_train,neighbourLabel)

new_label = utils.to_categorical(new_label)

new_model = build_LeNet5((28,28,1))
define_batch_size = round(new_train.shape[0] * 0.001)  # batch_size
print("\nTraining：")
new_model.fit(new_train, new_label, batch_size= define_batch_size, epochs= 3)

print("\nEvaluation：")
final_loss, final_accuracy = new_model.evaluate(x_test, y_test)
print("loss= ", final_loss)
print("accuracy= ", final_accuracy)
    
        
print("success!")
new_model.save('Model/lenet5_mnist_newdataset.h5')


Training：
Epoch 1/3
1003/1003 [==============================] - 37s 36ms/step - loss: 4.7544 - accuracy: 0.2282
Epoch 2/3
1003/1003 [==============================] - 37s 37ms/step - loss: 1.3406 - accuracy: 0.5116
Epoch 3/3
1003/1003 [==============================] - 36s 36ms/step - loss: 1.2060 - accuracy: 0.5572

Evaluation：
313/313 [==============================] - 1s 4ms/step - loss: 0.1756 - accuracy: 0.9436
loss=  0.17559798061847687
accuracy=  0.9435999989509583
success!


In [7]:
def neighbour_image(x_train,y_train,beta):
    
    mean = np.zeros(2)  
    cov = np.eye(2)
    neighbourlist = []
    neighbourLabel = []
    for i in range(50000):
        count = 0
        p = np.random.multivariate_normal(mean, cov,(28,28)) #sample formula
        p = p.reshape(-1,28,28,1)
        neighbour = p[0] + x_train[i]*255
        neighbour /= 255
        distance=np.linalg.norm(p[0]) 
        distance /= 255
        
        if (distance < beta or distance == beta):
            pred = lenet5.predict(neighbour.reshape(-1, 28, 28, 1),batch_size=128).argmax(axis=1)
            label = y_train[i]
            
            if(pred == label): 
                neighbour = (neighbour*255).astype(np.uint8).reshape(28,28,1)
                
                neighbourlist.append(neighbour)
                neighbourLabel.append(label)
                
                
            else:
                i = i-1
            
    return neighbourlist,neighbourLabel

In [5]:
def build_LeNet5(inputShape): #build model 

    lenet5 = Sequential()
    lenet5.add(Conv2D(6, (5, 5), padding="valid", input_shape=inputShape))
    lenet5.add(Activation("relu"))
    lenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    lenet5.add(Conv2D(16, (5, 5), padding="valid"))
    lenet5.add(Activation("relu"))
    lenet5.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    lenet5.add(Flatten())
    lenet5.add(Dense(120, activation= "relu"))
    lenet5.add(Dropout(0.5))
    lenet5.add(Dense(84, activation= "relu"))
    lenet5.add(Dropout(0.5))
    

    lenet5.add(Dense(10))
    lenet5.add(Activation("softmax"))
    
    lenet5.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
    
    return lenet5